<img src="https://pages.cnpem.br/workshopbioimagens/wp-content/uploads/sites/166/2023/06/logo-ilum-2048x382.png" alt="Descrição da imagem" style="width: 1000px; height: auto; ">

<div style=" padding: 10px; font-size: 34px; text-align: center;">
<strong>A Fera com classe</strong> 


<div style=" padding: 10px; font-size: 17px; text-align: center;">
<strong>Autor:</strong> Ana Luz Pereira Mendes 
<div style=" padding: 10px; font-size: 17px; text-align: center;">
<strong>Professor:</strong> Daniel R. Cassar

# 4.1 Quem classifica a classe classificadora?

Objetivo: altere a rede neural feita em Python puro para resolver um problema de
classificação. Treine uma rede neural em um dataset simples de classificação para mostrar
que funciona.

Comentário: aqui é necessário se informar sobre as diferenças de uma rede neural
classificadora com relação a uma rede neural regressora. A função de perda, por exemplo,
não poderá ser mais a função de perda dos resíduos quadrados.

Comentário 2: observe que o enunciado diz claramente que é para realizar a tarefa
na rede neural feita em Python puro nos vídeos da disciplina. Se você está usando o
PyTorch, numpy, tensorflow, keras, lightning ou qualquer outra biblioteca pronta,
você está no caminho errado!


-----------------

# Prelimináres

In [1]:
from graphviz import Digraph


def _tracar(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd
    """
    vertices = set()
    arestas = set()

    def construir(v):
        """Função recursiva para traçar o grafo."""
        if v not in vertices:
            vertices.add(v)
            for progenitor in v.progenitor:
                arestas.add((progenitor, v))
                construir(progenitor)

    construir(folha)

    return vertices, arestas


def plota_grafo(folha):
    """Função modificada da criada por Andrej Karpathy para construção de grafo.

    Referência: https://github.com/karpathy/micrograd

    """
    grafo = Digraph(format="svg", graph_attr={"rankdir": "LR"})
    vertices, arestas = _tracar(folha)

    for v in vertices:
        id_vertice = str(id(v))

        if hasattr(v, "rotulo") and (hasattr(v, "grad")):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f} | grad {v.grad:.3f}" + " }"

        elif hasattr(v, "rotulo"):
            texto = "{ " + f"{v.rotulo} | data {v.data:.3f}" + " }"

        else:
            texto = "{ " + f"data {v.data:.3f}" + " }"

        grafo.node(name=id_vertice, label=texto, shape="record")

        if v.operador_mae:
            grafo.node(name=id_vertice + v.operador_mae, label=v.operador_mae)
            grafo.edge(id_vertice + v.operador_mae, id_vertice)

    for vertice1, vertice2 in arestas:
        grafo.edge(str(id(vertice1)), str(id(vertice2)) + vertice2.operador_mae)

    return grafo

In [2]:
import math

class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()
    
    def log(self, base=math.e):
        
        assert self.data > 0 # log tem que ser valores positivos
        
        progenitor = (self, )
        data = math.log(self.data, base)
        operador_mae = f"log_{base}"
        resultado = Valor(data, progenitor, operador_mae)

        def propagar_log():
            self.grad += resultado.grad * (1 / (self.data * math.log(base))) # 1/x*ln(b)
        
        resultado.propagar = propagar_log

        return resultado
    
    def __lt__(self, other):

        if isinstance(other, (int, float)):
        
            return self.data < other

# Rede Neural

**Primeiro passo:** usar como função de ativação uma sigmoide.

In [3]:
import random

class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig() 
        
        return dado_de_saida # sai um único valor 
    
    def parametros(self):
        return self.pesos + [self.vies]

Criando as camadas...

In [4]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios: #cada neurônio roda os dados x (lista tipo(peso, altura)) e retorna um unico valor
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1: # quer dizer que tenho um único dado de saida
            return dados_de_saida[0] # logo posso retornar ele tirando do formato lista
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

**Segundo passo:** Converter as saidas para que retornem tanto a probabilidade quanto as classes.

- Mas como fazer isso?

Como utilizamos uma função sigmoide podemos converter seus valores de probabilidade a partir da regra:



\begin{cases} 
"verde", & \text{se } probabilidade > 0.5  \\
"velmelho", & \text{se } probabilidade \leq 0.5 
\end{cases}





In [8]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):

        percurso = [num_dados_entrada] + num_neuronios_por_camada# é bom lembrar que num_neuro por camada é uma lista
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i]) # em percurso[i] é o numero de dados de entrada que é justamente o numero de neurônios da camada anterior
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x,classes=[]):

        for camada in self.camadas:
            x = camada(x) # aqui vai entrar na primeira camada gerar um lista de valores que entra na proxima camada e gera mais uma lista até que gere apenas um único valor
        
        prob = x

        if prob < 0.5:
            return classes[1], prob

        else:
            return classes[0], prob
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

Vamos realizar teste a partir de dados artificiais. Observe:

In [9]:
x = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]

y_true = ["verde", "vermelho", "vermelo","verde"]


NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2] 
CLASSES = ["verde","vermelho"] 

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

In [10]:
pred1, prob = minha_mlp(x[0], classes=CLASSES)

print(pred1)
print(prob)

vermelho
Valor(data=0.4840038507639214)


**Tereceiro passo:** Agora para calcular cada perda é necessário usar a função de custo *binary cross-entropy*, que segue a seguinte formula:

$H_p(q) = - (1/N) \times \sum [y_i \times log(p(y_i)) + (1 - y_i) \times log(1 - p(y_i))]$


In [14]:
import numpy as np


NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.5

# Transformar os dados para binários
y_true_bi = []
for y in y_true:
    if y =="verde":
        y_true_bi.append(1)
    else:
        y_true_bi.append(0)

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    probabilidades = []
    for exemplo in x:
        previsao, prob = minha_mlp(exemplo,classes=["verde","vermelho"])
        probabilidades.append(prob)
        y_pred.append(previsao)

    # loss
    N = len(y_true_bi)
    loss = Valor(0)
    for i in range(N):

        prob_ = probabilidades[i]

        prob0_ = (prob_*-1) + 1

        loss += y_true_bi[i] * prob_.log(10) + (1 - y_true_bi[i]) * prob0_.log(10)

    loss = -loss / N

    print(type(loss))
    print(loss)
    
    # zero grad
    for p in minha_mlp.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    print(epoca, loss.data)

<class '__main__.Valor'>
Valor(data=0.28853643758269265)
0 0.28853643758269265
<class '__main__.Valor'>
Valor(data=0.2883868876912653)
1 0.2883868876912653
<class '__main__.Valor'>
Valor(data=0.28823531095547567)
2 0.28823531095547567
<class '__main__.Valor'>
Valor(data=0.2880816737263854)
3 0.2880816737263854
<class '__main__.Valor'>
Valor(data=0.28792594174032665)
4 0.28792594174032665
<class '__main__.Valor'>
Valor(data=0.2877680801071129)
5 0.2877680801071129
<class '__main__.Valor'>
Valor(data=0.2876080532980666)
6 0.2876080532980666
<class '__main__.Valor'>
Valor(data=0.28744582513386396)
7 0.28744582513386396
<class '__main__.Valor'>
Valor(data=0.2872813587721987)
8 0.2872813587721987
<class '__main__.Valor'>
Valor(data=0.287114616695263)
9 0.287114616695263
<class '__main__.Valor'>
Valor(data=0.28694556069705085)
10 0.28694556069705085
<class '__main__.Valor'>
Valor(data=0.28677415187048216)
11 0.28677415187048216
<class '__main__.Valor'>
Valor(data=0.286600350594351)
12 0.2866

In [15]:
print(y_true)
print(y_true_bi)

['verde', 'vermelho', 'vermelo', 'verde']
[1, 0, 0, 1]


In [16]:
print(y_pred)
print(probabilidades)

['verde', 'vermelho', 'vermelho', 'verde']
[Valor(data=0.7786104111299382), Valor(data=0.28480103776910337), Valor(data=0.3381450168351778), Valor(data=0.7208079538838489)]


# Conclusão
Podemos perceber que entendendo os diversos passos que uma rede neural possui temos a liberdade trabalhar com sua estrutura para resolver diferentes problemas. Esse trabalho foi interessante também para perceber que é necessário utilizar diferentes funções de perda para se adequar melhor ao problema em questão.

-----------------

# Referência

https://medium.com/ensina-ai/uma-explica%C3%A7%C3%A3o-visual-para-fun%C3%A7%C3%A3o-de-custo-binary-cross-entropy-ou-log-loss-eaee662c396c

https://medium.com/@msremigio/regress%C3%A3o-log%C3%ADstica-logistic-regression-997c6259ff9a